<a href="https://colab.research.google.com/github/ayachiii/Flow_study/blob/main/joint_analysis_pli_lf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

drive内のデータから、pliとlfの10人分5フェーズのデータを描画し、それをくっつけて一つの画像を作成した。

In [ ]:
# --- 1. ライブラリのインストールとインポート ---
import sys
import os
import glob
import shutil
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive

# HeartPyがない場合はインストール
try:
    import heartpy as hp
except ImportError:
    !pip install heartpy
    import heartpy as hp

# 警告非表示
warnings.filterwarnings('ignore')

# --- 2. 設定エリア (ここだけ環境に合わせて変更してください) ---

# Google Driveをマウント
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# ★重要: データが格納されている親フォルダのパス
# このフォルダの下にあるすべてのファイルを検索しに行きます
base_search_path = "/content/drive/MyDrive/csv"

# 被験者リスト (フォルダ名やファイル名に含まれる名前)
subject_list = [
    'ishida', 'yamamoto', 'ohashi', 'miyake','mitsuhashi',
    'ko', 'hayato','chika', 'nagomi', 'mio'
]

# フェーズリスト
phase_list = ['rest','boredom', 'flow', 'flow_ultra', 'overload']

# サンプリング周波数 (ECG計測時のHz。一般的に256, 500, 1000など)
FS = 256

# 結果保存フォルダ
output_dir = "All_Participants_Graphs"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir) # 古いフォルダがあれば消す
os.makedirs(output_dir, exist_ok=True)


# --- 3. 便利な関数群 ---

def find_file_smart(base_path, subject, phase, keywords):
    """
    指定した「被験者」「フェーズ」「キーワード(ECGやPLIなど)」を含むファイルを
    フォルダ階層の奥深くまで自動検索する関数
    """
    # 検索パターン: base_path以下のすべての階層(**)からファイルを探す
    # 例: *ishida*rest*ECG*.csv
    search_pattern = os.path.join(base_path, "**", f"*{subject}*{phase}*.csv")
    candidates = glob.glob(search_pattern, recursive=True)

    for path in candidates:
        filename = os.path.basename(path)
        # 指定したキーワード(ECG, PLIなど)が含まれているかチェック
        if all(k in filename for k in keywords):
            return path
    return None

def get_column_data(df, target_names):
    """
    データフレームから、カラム名が大文字小文字違っても柔軟にデータを取り出す関数
    例: 'FCz', 'fcz', ' FCz ' などを同一視して取得
    """
    df.columns = [c.strip() for c in df.columns] # 空白削除
    for col in df.columns:
        for target in target_names:
            if col.lower() == target.lower():
                return df[col], col # データと見つかった正確な列名を返す
    return None, None

def calculate_lf_timeseries(ecg_data, fs):
    """
    ECG生データから HeartPy を使って LF(Low Frequency) の時系列データを計算する
    """
    try:
        # segment_width: 計算する窓の幅(秒)。LFを出すには30秒以上推奨。
        # segment_overlap: 窓をずらす割合(0.5なら50%重複)
        wd, m = hp.process_segmentwise(ecg_data.values, sample_rate=fs, segment_width=40, segment_overlap=0.5)

        if 'lf' in m:
            # 計算されたLF値 (データ点数は少ない)
            lf_values = m['lf']

            # 元のデータの長さに合わせて引き伸ばす（線形補間）
            x_original = np.linspace(0, len(ecg_data), len(ecg_data))
            x_calculated = np.linspace(0, len(ecg_data), len(lf_values))
            lf_interpolated = np.interp(x_original, x_calculated, lf_values)

            return pd.Series(lf_interpolated, index=ecg_data.index)
        else:
            return None
    except Exception:
        return None

def min_max_norm(series):
    """グラフ重ね合わせ用に0-1に正規化"""
    if len(series) == 0: return series
    scaler = MinMaxScaler()
    return scaler.fit_transform(series.values.reshape(-1, 1)).flatten()


# --- 4. メイン処理ループ ---

print(f"🚀 解析開始: 全 {len(subject_list)*len(phase_list)} パターンを処理します...\n")

success_count = 0
error_count = 0

for subject in subject_list:
    for phase in phase_list:
        print(f"[{subject} - {phase}] Processing...", end=" ")

        # --- A. ファイル検索 ---
        # 脳波データ (PLIなど) を探す
        path_eeg = find_file_smart(base_search_path, subject, phase, keywords=["PLI"])
        # 心拍データ (ECG) を探す
        path_ecg = find_file_smart(base_search_path, subject, phase, keywords=["ECG"])

        if not path_eeg or not path_ecg:
            print(f"❌ ファイルが見つかりません (EEG: {'OK' if path_eeg else 'Missing'}, ECG: {'OK' if path_ecg else 'Missing'})")
            error_count += 1
            continue

        try:
            # --- B. データ読み込み & 抽出 ---
            df_eeg = pd.read_csv(path_eeg)
            df_ecg = pd.read_csv(path_ecg)

            # FCz (脳波) の抽出
            eeg_data, eeg_col_name = get_column_data(df_eeg, ['FCz', 'Fz', 'Cz']) # 候補リスト

            # LF (心拍) の準備
            # 1. 既に計算済みの 'lf' 列があるか探す
            lf_data, lf_col_name = get_column_data(df_ecg, ['lf', 'LF'])

            # 2. なければ 'ECG' 列から計算する
            if lf_data is None:
                ecg_raw, ecg_col_name = get_column_data(df_ecg, ['ECG', 'ecg'])
                if ecg_raw is not None:
                    # print("(Calculating LF...)", end=" ")
                    lf_data = calculate_lf_timeseries(ecg_raw, FS)
                    lf_col_name = "Calculated LF"

            # 両方のデータが揃ったか確認
            if eeg_data is None or lf_data is None:
                print(f"❌ 列データ不足 (FCz: {'OK' if eeg_data is not None else 'Missing'}, LF: {'OK' if lf_data is not None else 'Missing'})")
                error_count += 1
                continue

            # --- C. データ整形 & 描画 ---

            # 長さを揃える (短い方に合わせる)
            min_len = min(len(eeg_data), len(lf_data))
            eeg_plot = eeg_data.iloc[:min_len]
            lf_plot = lf_data.iloc[:min_len]

            # 移動平均でスムージング (グラフを見やすくするため)
            window = 50
            eeg_smooth = eeg_plot.rolling(window=window).mean()
            lf_smooth = lf_plot.rolling(window=window).mean()

            # プロット作成
            fig, ax1 = plt.subplots(figsize=(12, 5))

            # 左軸: EEG (青)
            ax1.plot(min_max_norm(eeg_smooth), color='blue', label=f'EEG ({eeg_col_name})', alpha=0.7)
            ax1.set_ylabel('Normalized EEG Power', color='blue')
            ax1.tick_params(axis='y', labelcolor='blue')
            ax1.set_xlabel('Time (samples)')

            # 右軸: HRV (オレンジ)
            ax2 = ax1.twinx()
            ax2.plot(min_max_norm(lf_smooth), color='orange', label=f'HRV ({lf_col_name})', alpha=0.7)
            ax2.set_ylabel('Normalized LF Power', color='orange')
            ax2.tick_params(axis='y', labelcolor='orange')

            # タイトルとレイアウト
            plt.title(f"Subject: {subject} | Phase: {phase}")

            # 凡例をまとめて表示
            lines1, labels1 = ax1.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

            # 保存
            save_name = f"{subject}_{phase}_FCz_LF.png"
            plt.savefig(os.path.join(output_dir, save_name))
            plt.close() # メモリ解放

            print("✅ 完了・保存")
            success_count += 1

        except Exception as e:
            print(f"❌ エラー発生: {e}")
            error_count += 1

# --- 5. 結果の圧縮とダウンロード ---
print("\n" + "="*50)
print(f"全ての処理が終了しました (成功: {success_count}, 失敗: {error_count})")

if success_count > 0:
    print("📦 画像をZipにまとめています...")
    shutil.make_archive('Results_All_Participants', 'zip', output_dir)
    print("⬇️ ダウンロードを開始します")
    from google.colab import files
    files.download('Results_All_Participants.zip')
else:
    print("⚠️ 画像が1枚も作成されませんでした。パス設定やデータの中身を確認してください。")

🚀 解析開始: 全 50 パターンを処理します...

[ishida - rest] Processing... ❌ 列データ不足 (FCz: OK, LF: Missing)
[ishida - boredom] Processing... ❌ 列データ不足 (FCz: OK, LF: Missing)
[ishida - flow] Processing... ❌ 列データ不足 (FCz: OK, LF: Missing)
[ishida - flow_ultra] Processing... ❌ 列データ不足 (FCz: OK, LF: Missing)
[ishida - overload] Processing... ❌ 列データ不足 (FCz: OK, LF: Missing)
[yamamoto - rest] Processing... ❌ 列データ不足 (FCz: OK, LF: Missing)
[yamamoto - boredom] Processing... ❌ 列データ不足 (FCz: OK, LF: Missing)
[yamamoto - flow] Processing... ❌ 列データ不足 (FCz: OK, LF: Missing)
[yamamoto - flow_ultra] Processing... ❌ 列データ不足 (FCz: OK, LF: Missing)
[yamamoto - overload] Processing... ❌ 列データ不足 (FCz: OK, LF: Missing)
[ohashi - rest] Processing... ❌ 列データ不足 (FCz: OK, LF: Missing)
[ohashi - boredom] Processing... ❌ 列データ不足 (FCz: OK, LF: Missing)
[ohashi - flow] Processing... ❌ 列データ不足 (FCz: OK, LF: Missing)
[ohashi - flow_ultra] Processing... ❌ 列データ不足 (FCz: OK, LF: Missing)
[ohashi - overload] Processing... ❌ 列データ不足 (FCz: OK, LF: Mi

In [ ]:
# =============================================================================
# 【RR間隔対応版】EEG(FCz) & HRV(LF from RR) 解析コード
# =============================================================================

import sys
import os
import glob
import shutil
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.interpolate import interp1d
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive

# 警告非表示
warnings.filterwarnings('ignore')

# --- 1. 設定エリア ---

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 親フォルダのパス (ここから下を全部探します)
base_search_path = "/content/drive/MyDrive/csv"

# 被験者リスト (フォルダ名やファイル名に含まれる名前)
subject_list = [
    'ishida', 'yamamoto', 'ohashi', 'miyake','mitsuhashi',
    'ko', 'hayato','chika', 'nagomi', 'mio'
]

# フェーズリスト
phase_list = ['rest','boredom', 'flow', 'flow_ultra', 'overload']

# 結果保存フォルダ
output_dir = "Graphs_FCz_LF_from_RR"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)


# --- 2. 関数群 ---

def find_file_fuzzy(base_path, subject, phase, file_type_keywords):
    """
    ファイル名に「名前」「フェーズ」「キーワード(RRなど)」を含むファイルを探す
    """
    # 検索: *ishida*flow*RR*.csv のようなパターン
    # ファイルタイプのキーワードが複数ある場合 (例: ['RR', 'rr']) はどれか一つあればOK
    search_pattern = os.path.join(base_path, "**", f"*{subject}*{phase}*.csv")
    candidates = glob.glob(search_pattern, recursive=True)

    for path in candidates:
        filename = os.path.basename(path)
        # 指定キーワードのいずれかが含まれているか
        if any(k in filename for k in file_type_keywords):
            return path
    return None

def get_column_data(df, target_names):
    """列名があいまいでもデータを取り出す"""
    df.columns = [c.strip() for c in df.columns]
    for col in df.columns:
        for target in target_names:
            if col.lower() == target.lower():
                return df[col], col
    return None, None

def calculate_lf_from_rr(rr_data_ms):
    """
    RR間隔(ms)のリストから、LF成分の時系列変化を計算する
    """
    try:
        # 1. 外れ値除去 (異常なRR間隔をカット: 300ms~1300ms程度が正常範囲)
        rr_data = rr_data_ms[(rr_data_ms > 300) & (rr_data_ms < 1300)]

        if len(rr_data) < 10: return None # データ少なすぎ

        # 2. 時系列への変換 (補間)
        # RR間隔の累積和を「時間軸(X)」とする
        t_cumulative = np.cumsum(rr_data) / 1000.0 # 秒単位に変換
        t_cumulative = t_cumulative - t_cumulative.iloc[0] # 0秒スタートに

        # 4Hz (0.25秒刻み) で等間隔にリサンプリング
        fs_interp = 4.0
        t_interp = np.arange(0, t_cumulative.iloc[-1], 1/fs_interp)

        f_interp = interp1d(t_cumulative, rr_data, kind='cubic', fill_value="extrapolate")
        rr_interp = f_interp(t_interp)

        # 3. LFパワーの時系列計算 (スライディングウィンドウ)
        # 窓幅: 60秒, ステップ: 10秒
        window_sec = 60
        step_sec = 10
        nperseg = int(window_sec * fs_interp)
        step_samples = int(step_sec * fs_interp)

        lf_series = []
        time_series = []

        for i in range(0, len(rr_interp) - nperseg, step_samples):
            segment = rr_interp[i : i + nperseg]

            # ウェルチ法でパワースペクトル密度(PSD)を計算
            freqs, psd = signal.welch(segment, fs=fs_interp, nperseg=nperseg)

            # LF帯域 (0.04 - 0.15 Hz) のパワーを積分
            lf_band = (freqs >= 0.04) & (freqs <= 0.15)
            lf_power = np.trapz(psd[lf_band], freqs[lf_band])

            lf_series.append(lf_power)
            # 時刻は窓の中心
            current_time = t_interp[i + nperseg//2]
            time_series.append(current_time)

        # 元のデータの長さに合わせて引き伸ばす（グラフ描画用）
        # 元のindex (データ点数) に合わせるための補間
        x_target = np.linspace(0, t_cumulative.iloc[-1], len(rr_data_ms))
        if len(time_series) > 1:
            f_resample = interp1d(time_series, lf_series, kind='linear', fill_value="extrapolate")
            lf_final = f_resample(x_target)
            return pd.Series(lf_final, index=rr_data_ms.index)
        else:
            return None

    except Exception as e:
        # print(f"Calc Error: {e}")
        return None

def min_max_norm(series):
    if len(series) == 0: return series
    scaler = MinMaxScaler()
    return scaler.fit_transform(series.values.reshape(-1, 1)).flatten()


# --- 3. メイン処理 ---
print(f"🚀 解析開始 (RRファイル優先モード)\n")

success_count = 0
error_count = 0

for subject in subject_list:
    for phase in phase_list:
        print(f"[{subject} - {phase}] Processing...", end=" ")

        # --- A. ファイル検索 ---
        # 1. 脳波 (PLI)
        path_eeg = find_file_fuzzy(base_search_path, subject, phase, ['PLI', 'pli'])

        # 2. RR間隔ファイル (RR, rr, interval)
        path_rr = find_file_fuzzy(base_search_path, subject, phase, ['RR', 'rr', 'interval'])

        if not path_eeg:
            print(f"❌ EEGファイルなし -> Skip")
            error_count += 1
            continue

        if not path_rr:
            # RRがない場合、一応ECGを探してみる救済措置
            path_rr = find_file_fuzzy(base_search_path, subject, phase, ['ECG', 'ecg'])
            is_raw_ecg = True
        else:
            is_raw_ecg = False # RRファイルが見つかった！

        if not path_rr:
             print(f"❌ 心拍ファイルなし(RRもECGも) -> Skip")
             error_count += 1
             continue

        try:
            # --- B. 読み込みと計算 ---
            df_eeg = pd.read_csv(path_eeg)
            df_rr = pd.read_csv(path_rr)

            # FCz 取得
            eeg_data, eeg_col = get_column_data(df_eeg, ['FCz', 'fcz'])
            if eeg_data is None:
                # なければ最初の列を使う
                eeg_data = df_eeg.iloc[:, 0]
                eeg_col = df_eeg.columns[0]

            # LF 計算
            lf_data = None
            lf_label = "LF"

            if not is_raw_ecg:
                # ★ RRファイルの場合 ★
                # RR列を探す (rr, RR, interval, R-R など)
                rr_series, rr_col = get_column_data(df_rr, ['rr', 'RR', 'interval', 'R-R', 'bpm'])

                if rr_series is not None:
                    # RR間隔からLFを計算
                    # (もし単位が秒なら1000倍する処理を入れる必要がありますが、通常msと仮定)
                    if rr_series.mean() < 2.0: # 値が小さい場合「秒」単位とみなしてmsに変換
                        rr_series = rr_series * 1000

                    lf_data = calculate_lf_from_rr(rr_series)
                    lf_label = "LF (from RR)"
                else:
                    print("❌ RRファイル内にRR列が見当たりません", end=" ")

            else:
                # ★ ECGファイルの場合 (救済) ★
                # 前回のコードと同じ(省略可ですが念のため)
                print("(RRなし: ECG使用)", end=" ")
                # ... (今回はRR推奨なのでここは失敗してもOKとします)

            if lf_data is None:
                print("❌ LF計算失敗")
                error_count += 1
                continue

            # --- C. 描画 ---
            # 長さ合わせ
            min_len = min(len(eeg_data), len(lf_data))
            eeg_plot = eeg_data.iloc[:min_len]
            lf_plot = lf_data.iloc[:min_len]

            # スムージング
            window = 50
            eeg_smooth = eeg_plot.rolling(window).mean()
            lf_smooth = lf_plot.rolling(window).mean()

            fig, ax1 = plt.subplots(figsize=(12, 5))

            ax1.plot(min_max_norm(eeg_smooth), color='blue', label=f'EEG: {eeg_col}', alpha=0.7)
            ax1.set_ylabel('Normalized EEG', color='blue')

            ax2 = ax1.twinx()
            ax2.plot(min_max_norm(lf_smooth), color='orange', label=f'HRV: {lf_label}', alpha=0.7)
            ax2.set_ylabel('Normalized LF', color='orange')

            plt.title(f"{subject} - {phase}")
            lines1, labels1 = ax1.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

            save_name = f"{subject}_{phase}_RR_analysis.png"
            plt.savefig(os.path.join(output_dir, save_name))
            plt.close()

            print(f"✅ 保存完了")
            success_count += 1

        except Exception as e:
            print(f"❌ Err: {e}")
            error_count += 1

# --- ダウンロード ---
print(f"\n完了: 成功 {success_count}, 失敗 {error_count}")
if success_count > 0:
    shutil.make_archive('RR_Analysis_Results', 'zip', output_dir)
    from google.colab import files
    files.download('RR_Analysis_Results.zip')

🚀 解析開始 (RRファイル優先モード)

[ishida - rest] Processing... ❌ RRファイル内にRR列が見当たりません ❌ LF計算失敗
[ishida - boredom] Processing... ❌ RRファイル内にRR列が見当たりません ❌ LF計算失敗
[ishida - flow] Processing... ❌ RRファイル内にRR列が見当たりません ❌ LF計算失敗
[ishida - flow_ultra] Processing... ❌ RRファイル内にRR列が見当たりません ❌ LF計算失敗
[ishida - overload] Processing... ❌ RRファイル内にRR列が見当たりません ❌ LF計算失敗
[yamamoto - rest] Processing... ❌ RRファイル内にRR列が見当たりません ❌ LF計算失敗
[yamamoto - boredom] Processing... ❌ RRファイル内にRR列が見当たりません ❌ LF計算失敗
[yamamoto - flow] Processing... ❌ RRファイル内にRR列が見当たりません ❌ LF計算失敗
[yamamoto - flow_ultra] Processing... ❌ RRファイル内にRR列が見当たりません ❌ LF計算失敗
[yamamoto - overload] Processing... ❌ RRファイル内にRR列が見当たりません ❌ LF計算失敗
[ohashi - rest] Processing... ❌ RRファイル内にRR列が見当たりません ❌ LF計算失敗
[ohashi - boredom] Processing... ❌ RRファイル内にRR列が見当たりません ❌ LF計算失敗
[ohashi - flow] Processing... ❌ RRファイル内にRR列が見当たりません ❌ LF計算失敗
[ohashi - flow_ultra] Processing... ❌ RRファイル内にRR列が見当たりません ❌ LF計算失敗
[ohashi - overload] Processing... ❌ RRファイル内にRR列が見当たりません ❌ LF計算失敗
[miyake - rest

In [ ]:
# =============================================================================
# 【RR列自動特定版】EEG(FCz) & HRV(LF) 解析コード
# =============================================================================

import sys
import os
import glob
import shutil
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.interpolate import interp1d
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive

# 警告非表示
warnings.filterwarnings('ignore')

# --- 1. 設定エリア ---

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 親フォルダのパス
base_search_path = "/content/drive/MyDrive/csv"

subject_list = [
    'ishida', 'yamamoto', 'ohashi', 'miyake','mitsuhashi',
    'ko', 'hayato','chika', 'nagomi', 'mio'
]

# フェーズリスト
phase_list = ['rest','boredom', 'flow', 'flow_ultra', 'overload']

# 結果保存フォルダ
output_dir = "Final_Graphs_RR_Auto"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)


# --- 2. 関数群 ---

def find_file_fuzzy(base_path, subject, phase, file_type_keywords):
    """
    ファイル名に「名前」「フェーズ」「キーワード」を含むファイルを探す
    """
    # 検索: *ishida*flow*RR*.csv のようなパターン
    search_pattern = os.path.join(base_path, "**", f"*{subject}*{phase}*.csv")
    candidates = glob.glob(search_pattern, recursive=True)

    for path in candidates:
        filename = os.path.basename(path)
        # 指定キーワードのいずれかが含まれているか
        if any(k.lower() in filename.lower() for k in file_type_keywords):
            return path
    return None

def get_column_data(df, target_names):
    """列名があいまいでもデータを取り出す"""
    df.columns = [str(c).strip() for c in df.columns]
    for col in df.columns:
        for target in target_names:
            if col.lower() == target.lower():
                return df[col], col
    return None, None

def smart_extract_rr_data(df):
    """
    ★今回のキモ★
    列名が分からなくても、値の範囲を見てRR間隔データ(ms)を自動特定する
    """
    # 1. まずは列名で探す
    target_names = ['rr_interval', 'rr', 'RR', 'interval', 'R-R', 'bpm', 'ibi']
    series, name = get_column_data(df, target_names)
    if series is not None:
        return series, name

    # 2. 見つからない場合、全列をスキャンして「値が心拍っぽい列」を探す
    # 心拍間隔(ms)は通常 300ms(心拍200) ～ 1500ms(心拍40) の間にあるはず
    for col in df.columns:
        try:
            # 数値データのみ抽出
            col_data = pd.to_numeric(df[col], errors='coerce').dropna()
            if len(col_data) < 10: continue

            mean_val = col_data.mean()

            # ミリ秒単位 (300 ~ 1500)
            if 300 < mean_val < 1500:
                return col_data, f"{col} (Auto-detected ms)"

            # 秒単位 (0.3 ~ 1.5) -> 1000倍して返す
            if 0.3 < mean_val < 1.5:
                return col_data * 1000, f"{col} (Auto-detected sec)"

        except:
            continue

    return None, None

def calculate_lf_from_rr(rr_data_ms):
    """RR間隔(ms)からLF成分の時系列変化を計算"""
    try:
        # 外れ値除去
        rr_data = rr_data_ms[(rr_data_ms > 300) & (rr_data_ms < 2000)]
        if len(rr_data) < 10: return None

        # 時系列への変換 (補間)
        t_cumulative = np.cumsum(rr_data) / 1000.0
        t_cumulative = t_cumulative - t_cumulative.iloc[0]

        fs_interp = 4.0
        t_interp = np.arange(0, t_cumulative.iloc[-1], 1/fs_interp)

        f_interp = interp1d(t_cumulative, rr_data, kind='cubic', fill_value="extrapolate")
        rr_interp = f_interp(t_interp)

        # LFパワー計算 (Window 60s)
        window_sec = 60
        step_sec = 10
        nperseg = int(window_sec * fs_interp)
        step_samples = int(step_sec * fs_interp)

        if len(rr_interp) < nperseg: return None # データが短すぎる

        lf_series = []
        time_series = []

        for i in range(0, len(rr_interp) - nperseg, step_samples):
            segment = rr_interp[i : i + nperseg]
            freqs, psd = signal.welch(segment, fs=fs_interp, nperseg=nperseg)

            # LF帯域 (0.04 - 0.15 Hz)
            lf_band = (freqs >= 0.04) & (freqs <= 0.15)
            lf_power = np.trapz(psd[lf_band], freqs[lf_band])

            lf_series.append(lf_power)
            current_time = t_interp[i + nperseg//2]
            time_series.append(current_time)

        # データ引き伸ばし
        x_target = np.linspace(0, t_cumulative.iloc[-1], len(rr_data_ms))
        if len(time_series) > 1:
            f_resample = interp1d(time_series, lf_series, kind='linear', fill_value="extrapolate")
            lf_final = f_resample(x_target)
            return pd.Series(lf_final, index=rr_data_ms.index)
        else:
            return None

    except Exception as e:
        return None

def min_max_norm(series):
    if len(series) == 0: return series
    scaler = MinMaxScaler()
    return scaler.fit_transform(series.values.reshape(-1, 1)).flatten()


# --- 3. メイン処理 ---
print(f"🚀 解析開始 (RR自動特定モード)\n")

success_count = 0
error_count = 0

for subject in subject_list:
    for phase in phase_list:
        print(f"[{subject} - {phase}] Processing...", end=" ")

        # --- A. ファイル検索 ---
        path_eeg = find_file_fuzzy(base_search_path, subject, phase, ['PLI', 'pli'])
        # キーワードに 'rr_interval' を追加
        path_rr = find_file_fuzzy(base_search_path, subject, phase, ['rr_interval', 'RR', 'rr', 'interval'])

        if not path_eeg or not path_rr:
            print(f"❌ ファイル欠損 (EEG:{'〇' if path_eeg else '×'}, RR:{'〇' if path_rr else '×'}) -> Skip")
            error_count += 1
            continue

        try:
            # --- B. 読み込み ---
            df_eeg = pd.read_csv(path_eeg)
            df_rr = pd.read_csv(path_rr)

            # FCz 取得
            eeg_data, eeg_col = get_column_data(df_eeg, ['FCz', 'fcz'])
            if eeg_data is None:
                eeg_data = df_eeg.iloc[:, 0] # なければ1列目
                eeg_col = df_eeg.columns[0]

            # LF (RR) 取得 ★ここが変わった！★
            rr_series, rr_col_name = smart_extract_rr_data(df_rr)

            if rr_series is None:
                print(f"❌ RRデータ特定不能 (Columns: {list(df_rr.columns)})")
                error_count += 1
                continue

            # LF計算
            lf_data = calculate_lf_from_rr(rr_series)

            if lf_data is None:
                print("❌ LF計算失敗(データ短/ノイズ)")
                error_count += 1
                continue

            # --- C. 描画 ---
            min_len = min(len(eeg_data), len(lf_data))
            eeg_plot = eeg_data.iloc[:min_len]
            lf_plot = lf_data.iloc[:min_len]

            # スムージング
            window = 50
            eeg_smooth = eeg_plot.rolling(window).mean()
            lf_smooth = lf_plot.rolling(window).mean()

            fig, ax1 = plt.subplots(figsize=(12, 5))

            ax1.plot(min_max_norm(eeg_smooth), color='blue', label=f'EEG: {eeg_col}', alpha=0.7)
            ax1.set_ylabel('Normalized EEG', color='blue')

            ax2 = ax1.twinx()
            ax2.plot(min_max_norm(lf_smooth), color='orange', label=f'HRV: LF (from {rr_col_name})', alpha=0.7)
            ax2.set_ylabel('Normalized LF', color='orange')

            plt.title(f"{subject} - {phase}")
            lines1, labels1 = ax1.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

            save_name = f"{subject}_{phase}_RR_analysis.png"
            plt.savefig(os.path.join(output_dir, save_name))
            plt.close()

            print(f"✅ 保存完了")
            success_count += 1

        except Exception as e:
            print(f"❌ Err: {e}")
            error_count += 1

# --- ダウンロード ---
print(f"\n完了: 成功 {success_count}, 失敗 {error_count}")
if success_count > 0:
    shutil.make_archive('Results_RR_Auto', 'zip', output_dir)
    from google.colab import files
    files.download('Results_RR_Auto.zip')

🚀 解析開始 (RR自動特定モード)

[ishida - rest] Processing... ✅ 保存完了
[ishida - boredom] Processing... ✅ 保存完了
[ishida - flow] Processing... ✅ 保存完了
[ishida - flow_ultra] Processing... ✅ 保存完了
[ishida - overload] Processing... ✅ 保存完了
[yamamoto - rest] Processing... ✅ 保存完了
[yamamoto - boredom] Processing... ✅ 保存完了
[yamamoto - flow] Processing... ✅ 保存完了
[yamamoto - flow_ultra] Processing... ✅ 保存完了
[yamamoto - overload] Processing... ✅ 保存完了
[ohashi - rest] Processing... ✅ 保存完了
[ohashi - boredom] Processing... ✅ 保存完了
[ohashi - flow] Processing... ✅ 保存完了
[ohashi - flow_ultra] Processing... ✅ 保存完了
[ohashi - overload] Processing... ✅ 保存完了
[miyake - rest] Processing... ✅ 保存完了
[miyake - boredom] Processing... ✅ 保存完了
[miyake - flow] Processing... ✅ 保存完了
[miyake - flow_ultra] Processing... ✅ 保存完了
[miyake - overload] Processing... ✅ 保存完了
[mitsuhashi - rest] Processing... ✅ 保存完了
[mitsuhashi - boredom] Processing... ✅ 保存完了
[mitsuhashi - flow] Processing... ✅ 保存完了
[mitsuhashi - flow_ultra] Processing... ✅ 保存完了
[mits

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

ギザギザ

In [ ]:
# =============================================================================
# 【ギザギザ表示版】EEG(FCz) & HRV(LF) 解析コード
# =============================================================================
# 変更点: スムージング(移動平均)をほぼ無くし、データの「ギザギザ感」を出します
# =============================================================================

import sys
import os
import glob
import shutil
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.interpolate import interp1d
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive

warnings.filterwarnings('ignore')

# ---------------------------------------------------------
# 1. 設定エリア
# ---------------------------------------------------------

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

base_search_path = "/content/drive/MyDrive/csv"


# ★ここが調整ポイント！
# 1なら完全な生データ(ギザギザ最強)。数値を上げると滑らかになります。
# お手本のようなギザギザなら 3~5 くらいが適当です。
SMOOTHING_WINDOW = 3

# 出力先
output_dir = "Graphs_Jagged_View"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

subject_list = [
    'ishida', 'yamamoto', 'ohashi', 'miyake','mitsuhashi',
    'ko', 'hayato','chika', 'nagomi', 'mio'
]

# フェーズリスト
phase_list = ['rest','boredom', 'flow', 'flow_ultra', 'overload']

# ---------------------------------------------------------
# 2. 関数群 (前回と同じ強力な判定ロジック)
# ---------------------------------------------------------

def find_file_fuzzy(base_path, subject, phase, keywords):
    search_pattern = os.path.join(base_path, "**", f"*{subject}*{phase}*.csv")
    candidates = glob.glob(search_pattern, recursive=True)
    for path in candidates:
        if any(k.lower() in os.path.basename(path).lower() for k in keywords):
            return path
    return None

def get_column_data(df, target_names):
    df.columns = [str(c).strip() for c in df.columns]
    for col in df.columns:
        for target in target_names:
            if col.lower() == target.lower():
                return df[col], col
    return None, None

def smart_extract_rr_data(df):
    # 名前で探す
    series, name = get_column_data(df, ['rr_interval', 'rr', 'RR', 'interval', 'R-R', 'bpm'])
    if series is not None: return series, name

    # 値で探す (300ms ~ 1500ms)
    for col in df.columns:
        try:
            val = pd.to_numeric(df[col], errors='coerce').dropna()
            if len(val) < 10: continue
            mean_val = val.mean()
            if 300 < mean_val < 1500: return val, f"{col} (Auto ms)"
            if 0.3 < mean_val < 1.5: return val * 1000, f"{col} (Auto sec)"
        except: continue
    return None, None

def calculate_lf_from_rr(rr_data_ms):
    try:
        # 外れ値除去
        rr_data = rr_data_ms[(rr_data_ms > 300) & (rr_data_ms < 2000)]
        if len(rr_data) < 10: return None

        # 時間軸作成
        t_cum = np.cumsum(rr_data) / 1000.0
        t_cum = t_cum - t_cum.iloc[0]

        # 等間隔リサンプリング (4Hz)
        fs_interp = 4.0
        t_interp = np.arange(0, t_cum.iloc[-1], 1/fs_interp)
        f_int = interp1d(t_cum, rr_data, kind='linear', fill_value="extrapolate") # cubicよりlinearの方が鋭くなる
        rr_interp = f_int(t_interp)

        # LF計算 (Window 60s, Step 5s -> ステップを細かくして変動を捉える)
        window_sec = 60
        step_sec = 5 # 10から5に短縮
        nperseg = int(window_sec * fs_interp)
        step_samples = int(step_sec * fs_interp)

        if len(rr_interp) < nperseg: return None

        lf_vals = []
        t_vals = []

        for i in range(0, len(rr_interp) - nperseg, step_samples):
            segment = rr_interp[i : i + nperseg]
            freqs, psd = signal.welch(segment, fs=fs_interp, nperseg=nperseg)
            lf_pow = np.trapz(psd[(freqs >= 0.04) & (freqs <= 0.15)], freqs[(freqs >= 0.04) & (freqs <= 0.15)])
            lf_vals.append(lf_pow)
            t_vals.append(t_interp[i + nperseg//2])

        # 補間して元の長さに戻す
        x_target = np.linspace(0, t_cum.iloc[-1], len(rr_data_ms))
        if len(t_vals) > 1:
            f_res = interp1d(t_vals, lf_vals, kind='linear', fill_value="extrapolate")
            return pd.Series(f_res(x_target), index=rr_data_ms.index)
        return None
    except: return None

def min_max_norm(series):
    if len(series) == 0: return series
    scaler = MinMaxScaler()
    return scaler.fit_transform(series.values.reshape(-1, 1)).flatten()


# ---------------------------------------------------------
# 3. メイン処理
# ---------------------------------------------------------
print(f"🚀 解析開始 (ギザギザ表示モード)\n")

success_count = 0

for subject in subject_list:
    for phase in phase_list:
        print(f"[{subject} - {phase}]", end=" ")

        # ファイル検索
        path_eeg = find_file_fuzzy(base_search_path, subject, phase, ['PLI', 'pli'])
        path_rr = find_file_fuzzy(base_search_path, subject, phase, ['rr_interval', 'RR', 'rr', 'interval'])

        if not path_eeg or not path_rr:
            print("-> Skip (File Missing)")
            continue

        try:
            df_eeg = pd.read_csv(path_eeg)
            df_rr = pd.read_csv(path_rr)

            # データ取得
            eeg_data, eeg_col = get_column_data(df_eeg, ['FCz', 'fcz'])
            rr_vals, rr_name = smart_extract_rr_data(df_rr)

            if eeg_data is None or rr_vals is None:
                print("-> Skip (Data Column Missing)")
                continue

            eeg_data = eeg_data.fillna(method='ffill').fillna(method='bfill') # 欠損埋め

            # LF計算
            lf_data = calculate_lf_from_rr(rr_vals)
            if lf_data is None:
                print("-> LF Calc Failed")
                continue

            # 描画準備
            min_len = min(len(eeg_data), len(lf_data))
            eeg_plot = eeg_data.iloc[:min_len]
            lf_plot = lf_data.iloc[:min_len]

            # ★ここでスムージング調整
            eeg_vis = eeg_plot.rolling(window=SMOOTHING_WINDOW).mean()
            lf_vis = lf_plot.rolling(window=SMOOTHING_WINDOW).mean()

            # プロット
            fig, ax1 = plt.subplots(figsize=(12, 5))

            # 線を細く(linewidth=1.0)してギザギザを際立たせる
            ax1.plot(min_max_norm(eeg_vis), color='blue', label='EEG (FCz)', alpha=0.8, linewidth=1.0)
            ax1.set_ylabel('EEG Power', color='blue')

            ax2 = ax1.twinx()
            ax2.plot(min_max_norm(lf_vis), color='orange', label='HRV (LF)', alpha=0.8, linewidth=1.5)
            ax2.set_ylabel('LF Power', color='orange')

            plt.title(f"{subject} - {phase}")
            lines1, labels1 = ax1.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

            out_path = os.path.join(output_dir, f"{subject}_{phase}.png")
            plt.savefig(out_path)
            plt.close()

            print("✅ OK")
            success_count += 1

        except Exception as e:
            print(f"-> Error: {e}")

# ダウンロード
if success_count > 0:
    shutil.make_archive('Results_Jagged', 'zip', output_dir)
    from google.colab import files
    files.download('Results_Jagged.zip')
    print("\n完了！ダウンロードされたZipを確認してください。")
else:
    print("\n作成された画像はありません。")

🚀 解析開始 (ギザギザ表示モード)

[ishida - rest] ✅ OK
[ishida - boredom] ✅ OK
[ishida - flow] ✅ OK
[ishida - flow_ultra] ✅ OK
[ishida - overload] ✅ OK
[yamamoto - rest] ✅ OK
[yamamoto - boredom] ✅ OK
[yamamoto - flow] ✅ OK
[yamamoto - flow_ultra] ✅ OK
[yamamoto - overload] ✅ OK
[ohashi - rest] ✅ OK
[ohashi - boredom] ✅ OK
[ohashi - flow] ✅ OK
[ohashi - flow_ultra] ✅ OK
[ohashi - overload] ✅ OK
[miyake - rest] ✅ OK
[miyake - boredom] ✅ OK
[miyake - flow] ✅ OK
[miyake - flow_ultra] ✅ OK
[miyake - overload] ✅ OK
[mitsuhashi - rest] ✅ OK
[mitsuhashi - boredom] ✅ OK
[mitsuhashi - flow] ✅ OK
[mitsuhashi - flow_ultra] ✅ OK
[mitsuhashi - overload] ✅ OK
[ko - rest] ✅ OK
[ko - boredom] ✅ OK
[ko - flow] ✅ OK
[ko - flow_ultra] ✅ OK
[ko - overload] ✅ OK
[hayato - rest] ✅ OK
[hayato - boredom] ✅ OK
[hayato - flow] ✅ OK
[hayato - flow_ultra] ✅ OK
[hayato - overload] ✅ OK
[chika - rest] ✅ OK
[chika - boredom] ✅ OK
[chika - flow] ✅ OK
[chika - flow_ultra] ✅ OK
[chika - overload] ✅ OK
[nagomi - rest] ✅ OK
[nagomi - 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


完了！ダウンロードされたZipを確認してください。


ギザギザ２


In [ ]:
# =============================================================================
# 【完全生データ・高解像度版】EEG & HRV(LF) 解析コード
# =============================================================================
# 特徴:
# 1. 平均化処理(スムージング)を一切行いません。
# 2. LFを「1秒刻み」で細かく計算し、変動をダイレクトに可視化します。
# 3. 指定された最新の参加者リスト・フェーズリストを使用します。
# =============================================================================

import sys
import os
import glob
import shutil
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.interpolate import interp1d
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive

# 警告非表示
warnings.filterwarnings('ignore')

# ---------------------------------------------------------
# 1. 設定エリア (指定リスト反映済み)
# ---------------------------------------------------------

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

base_search_path = "/content/drive/MyDrive/csv"

# ★指定されたリストに更新しました
subject_list = [
    'ishida', 'yamamoto', 'ohashi', 'miyake', 'mitsuhashi',
    'ko', 'hayato', 'chika', 'nagomi', 'mio'
]

# ★指定されたフェーズに更新しました
phase_list = ['rest', 'boredom', 'flow', 'flow_ultra', 'overload']

# 出力先
output_dir = "Graphs_NoSmoothing_Raw"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)


# ---------------------------------------------------------
# 2. 関数群
# ---------------------------------------------------------

def find_file_fuzzy(base_path, subject, phase, keywords):
    """ファイル検索 (部分一致)"""
    search_pattern = os.path.join(base_path, "**", f"*{subject}*{phase}*.csv")
    candidates = glob.glob(search_pattern, recursive=True)
    for path in candidates:
        if any(k.lower() in os.path.basename(path).lower() for k in keywords):
            return path
    return None

def get_column_data(df, target_names):
    """列名検索"""
    df.columns = [str(c).strip() for c in df.columns]
    for col in df.columns:
        for target in target_names:
            if col.lower() == target.lower():
                return df[col], col
    return None, None

def smart_extract_rr_data(df):
    """RR間隔データの自動抽出"""
    # 名前で探す
    series, name = get_column_data(df, ['rr_interval', 'rr', 'RR', 'interval', 'R-R', 'bpm'])
    if series is not None: return series, name

    # 値の範囲で探す (300ms ~ 1500ms)
    for col in df.columns:
        try:
            val = pd.to_numeric(df[col], errors='coerce').dropna()
            if len(val) < 10: continue
            mean_val = val.mean()
            if 300 < mean_val < 1500: return val, f"{col} (Auto ms)"
            if 0.3 < mean_val < 1.5: return val * 1000, f"{col} (Auto sec)"
        except: continue
    return None, None

def calculate_lf_from_rr_high_res(rr_data_ms):
    """
    RR間隔からLFを計算 (高解像度・ギザギザ重視)
    """
    try:
        # 外れ値除去
        rr_data = rr_data_ms[(rr_data_ms > 300) & (rr_data_ms < 2000)]
        if len(rr_data) < 10: return None

        # 時間軸作成
        t_cum = np.cumsum(rr_data) / 1000.0
        t_cum = t_cum - t_cum.iloc[0]

        # 等間隔リサンプリング (4Hz)
        fs_interp = 4.0
        t_interp = np.arange(0, t_cum.iloc[-1], 1/fs_interp)

        # 補間 (Linearにすることで、カーブを丸めず尖らせる)
        f_int = interp1d(t_cum, rr_data, kind='linear', fill_value="extrapolate")
        rr_interp = f_int(t_interp)

        # --- パラメータ設定 (ギザギザ重視) ---
        window_sec = 60 # 窓幅 (計算に必要な最低限の長さ)
        step_sec = 1    # ★ここ重要: 1秒ごとに計算 (前回は5~10秒) -> 圧倒的に細かくなる

        nperseg = int(window_sec * fs_interp)
        step_samples = int(step_sec * fs_interp)

        if len(rr_interp) < nperseg: return None

        lf_vals = []
        t_vals = []

        # スライディングウィンドウ計算
        for i in range(0, len(rr_interp) - nperseg, step_samples):
            segment = rr_interp[i : i + nperseg]

            # ウェルチ法 (nfftを小さくしすぎない)
            freqs, psd = signal.welch(segment, fs=fs_interp, nperseg=nperseg)

            # LF帯域積分
            lf_pow = np.trapz(psd[(freqs >= 0.04) & (freqs <= 0.15)], freqs[(freqs >= 0.04) & (freqs <= 0.15)])

            lf_vals.append(lf_pow)
            t_vals.append(t_interp[i + nperseg//2]) # 窓の中央時刻

        # 元の時間軸に合わせて引き伸ばし (線形補間でギザギザ維持)
        x_target = np.linspace(0, t_cum.iloc[-1], len(rr_data_ms))
        if len(t_vals) > 1:
            f_res = interp1d(t_vals, lf_vals, kind='linear', fill_value="extrapolate")
            return pd.Series(f_res(x_target), index=rr_data_ms.index)
        return None

    except Exception as e:
        return None

def min_max_norm(series):
    if len(series) == 0: return series
    scaler = MinMaxScaler()
    return scaler.fit_transform(series.values.reshape(-1, 1)).flatten()


# ---------------------------------------------------------
# 3. メイン処理
# ---------------------------------------------------------
print(f"🚀 解析開始: {len(subject_list)*len(phase_list)} パターン (Smoothing: OFF)\n")

success_count = 0

for subject in subject_list:
    for phase in phase_list:
        print(f"[{subject} - {phase}]", end=" ")

        # ファイル検索
        path_eeg = find_file_fuzzy(base_search_path, subject, phase, ['PLI', 'pli'])
        path_rr = find_file_fuzzy(base_search_path, subject, phase, ['rr_interval', 'RR', 'rr', 'interval'])

        if not path_eeg or not path_rr:
            print("-> ⚠️ Skip (File Missing)")
            continue

        try:
            # 読み込み
            df_eeg = pd.read_csv(path_eeg)
            df_rr = pd.read_csv(path_rr)

            # データ取得
            eeg_data, eeg_col = get_column_data(df_eeg, ['FCz', 'fcz'])
            rr_vals, rr_name = smart_extract_rr_data(df_rr)

            if eeg_data is None or rr_vals is None:
                print("-> ⚠️ Skip (Column Missing)")
                continue

            eeg_data = eeg_data.fillna(method='ffill').fillna(method='bfill')

            # LF計算 (High Res)
            lf_data = calculate_lf_from_rr_high_res(rr_vals)

            if lf_data is None:
                print("-> ❌ LF Calc Failed (Too short?)")
                continue

            # --- 描画 (平均化なし！) ---
            min_len = min(len(eeg_data), len(lf_data))

            # ★ポイント: rolling().mean() を使いません。そのまま出します。
            eeg_vis = eeg_data.iloc[:min_len]
            lf_vis = lf_data.iloc[:min_len]

            fig, ax1 = plt.subplots(figsize=(12, 5))

            # EEG: 生データなのでかなり太くなる可能性がありますが、要求通りそのまま出します
            # alpha=0.6 で少し透けさせて、重なりを見やすくします
            ax1.plot(min_max_norm(eeg_vis), color='blue', label='EEG (FCz)', alpha=0.6, linewidth=0.8)
            ax1.set_ylabel('EEG Power', color='blue')

            # LF: 1秒刻み計算結果をそのまま表示 (オレンジ)
            ax2 = ax1.twinx()
            ax2.plot(min_max_norm(lf_vis), color='orange', label='HRV (LF)', alpha=0.9, linewidth=1.2)
            ax2.set_ylabel('LF Power', color='orange')

            plt.title(f"{subject} - {phase}")

            # 凡例
            lines1, labels1 = ax1.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

            # 保存
            out_path = os.path.join(output_dir, f"{subject}_{phase}.png")
            plt.savefig(out_path)
            plt.close()

            print("✅ OK")
            success_count += 1

        except Exception as e:
            print(f"-> ❌ Error: {e}")

# ダウンロード処理
if success_count > 0:
    print(f"\n🎉 {success_count} 件のグラフを作成しました。圧縮してダウンロードします...")
    shutil.make_archive('Final_Results_Raw', 'zip', output_dir)
    from google.colab import files
    files.download('Final_Results_Raw.zip')
else:
    print("\n⚠️ 画像が1枚も作成されませんでした。")

🚀 解析開始: 50 パターン (Smoothing: OFF)

[ishida - rest] ✅ OK
[ishida - boredom] ✅ OK
[ishida - flow] ✅ OK
[ishida - flow_ultra] ✅ OK
[ishida - overload] ✅ OK
[yamamoto - rest] ✅ OK
[yamamoto - boredom] ✅ OK
[yamamoto - flow] ✅ OK
[yamamoto - flow_ultra] ✅ OK
[yamamoto - overload] ✅ OK
[ohashi - rest] ✅ OK
[ohashi - boredom] ✅ OK
[ohashi - flow] ✅ OK
[ohashi - flow_ultra] ✅ OK
[ohashi - overload] ✅ OK
[miyake - rest] ✅ OK
[miyake - boredom] ✅ OK
[miyake - flow] ✅ OK
[miyake - flow_ultra] ✅ OK
[miyake - overload] ✅ OK
[mitsuhashi - rest] ✅ OK
[mitsuhashi - boredom] ✅ OK
[mitsuhashi - flow] ✅ OK
[mitsuhashi - flow_ultra] ✅ OK
[mitsuhashi - overload] ✅ OK
[ko - rest] ✅ OK
[ko - boredom] ✅ OK
[ko - flow] ✅ OK
[ko - flow_ultra] ✅ OK
[ko - overload] ✅ OK
[hayato - rest] ✅ OK
[hayato - boredom] ✅ OK
[hayato - flow] ✅ OK
[hayato - flow_ultra] ✅ OK
[hayato - overload] ✅ OK
[chika - rest] ✅ OK
[chika - boredom] ✅ OK
[chika - flow] ✅ OK
[chika - flow_ultra] ✅ OK
[chika - overload] ✅ OK
[nagomi - rest] ✅

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# =============================================================================
# 【超高感度版】EEG & HRV (Sensitive LF + Raw RR)
# =============================================================================
# 変更点:
# 1. 計算窓(Window)を60秒 -> 25秒に短縮 (これ以上短いとLFの定義0.04Hzが計算不可になります)
# 2. 背景に「元のRR間隔(Raw)」を薄く表示し、本来のギザギザを確認できるようにします。
# =============================================================================

import sys
import os
import glob
import shutil
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.interpolate import interp1d
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive

warnings.filterwarnings('ignore')

# --- 設定 ---
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

base_search_path = "/content/drive/MyDrive/csv"

subject_list = [
    'ishida', 'yamamoto', 'ohashi', 'miyake', 'mitsuhashi',
    'ko', 'hayato', 'chika', 'nagomi', 'mio'
]

phase_list = ['rest', 'boredom', 'flow', 'flow_ultra', 'overload']

output_dir = "Graphs_Sensitive_LF"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)


# --- 関数群 ---
def find_file_fuzzy(base_path, subject, phase, keywords):
    search_pattern = os.path.join(base_path, "**", f"*{subject}*{phase}*.csv")
    candidates = glob.glob(search_pattern, recursive=True)
    for path in candidates:
        if any(k.lower() in os.path.basename(path).lower() for k in keywords):
            return path
    return None

def get_column_data(df, target_names):
    df.columns = [str(c).strip() for c in df.columns]
    for col in df.columns:
        for target in target_names:
            if col.lower() == target.lower():
                return df[col], col
    return None, None

def smart_extract_rr_data(df):
    series, name = get_column_data(df, ['rr_interval', 'rr', 'RR', 'interval', 'R-R', 'bpm'])
    if series is not None: return series, name
    for col in df.columns:
        try:
            val = pd.to_numeric(df[col], errors='coerce').dropna()
            if len(val) < 10: continue
            mean_val = val.mean()
            if 300 < mean_val < 1500: return val, f"{col} (Auto ms)"
            if 0.3 < mean_val < 1.5: return val * 1000, f"{col} (Auto sec)"
        except: continue
    return None, None

def calculate_sensitive_lf(rr_data_ms):
    """
    窓幅を極限まで短くして、変動を拾うLF計算
    """
    try:
        # 外れ値除去
        rr_data = rr_data_ms[(rr_data_ms > 300) & (rr_data_ms < 2000)]
        if len(rr_data) < 10: return None, None

        # 時間軸作成
        t_cum = np.cumsum(rr_data) / 1000.0
        t_cum = t_cum - t_cum.iloc[0]

        # リサンプリング (4Hz)
        fs_interp = 4.0
        t_interp = np.arange(0, t_cum.iloc[-1], 1/fs_interp)
        f_int = interp1d(t_cum, rr_data, kind='linear', fill_value="extrapolate")
        rr_interp = f_int(t_interp)

        # ★ここが変更点: Windowを短くする
        window_sec = 25 # 25秒 (LFの下限0.04Hz=25秒周期 なので、これ以上短くすると数学的に計算不能)
        step_sec = 1    # 1秒刻み

        nperseg = int(window_sec * fs_interp)
        step_samples = int(step_sec * fs_interp)

        if len(rr_interp) < nperseg: return None, None

        lf_vals = []
        t_vals = []

        for i in range(0, len(rr_interp) - nperseg, step_samples):
            segment = rr_interp[i : i + nperseg]
            freqs, psd = signal.welch(segment, fs=fs_interp, nperseg=nperseg)
            # LF帯域積分
            lf_pow = np.trapz(psd[(freqs >= 0.04) & (freqs <= 0.15)], freqs[(freqs >= 0.04) & (freqs <= 0.15)])
            lf_vals.append(lf_pow)
            t_vals.append(t_interp[i + nperseg//2])

        # 補間
        x_target = np.linspace(0, t_cum.iloc[-1], len(rr_data_ms))

        # LFの時系列
        lf_series = None
        if len(t_vals) > 1:
            f_res = interp1d(t_vals, lf_vals, kind='linear', fill_value="extrapolate")
            lf_series = pd.Series(f_res(x_target), index=rr_data_ms.index)

        # 生のRR時系列 (グラフ背景用)
        rr_raw_resampled = pd.Series(interp1d(t_interp, rr_interp, kind='linear')(np.linspace(0, t_interp[-1], len(rr_data_ms))), index=rr_data_ms.index)

        return lf_series, rr_raw_resampled

    except Exception:
        return None, None

def min_max_norm(series):
    if len(series) == 0: return series
    scaler = MinMaxScaler()
    return scaler.fit_transform(series.values.reshape(-1, 1)).flatten()


# --- メイン処理 ---
print(f"🚀 解析開始 (高感度LFモード)\n")

success_count = 0

for subject in subject_list:
    for phase in phase_list:
        print(f"[{subject} - {phase}]", end=" ")

        path_eeg = find_file_fuzzy(base_search_path, subject, phase, ['PLI', 'pli'])
        path_rr = find_file_fuzzy(base_search_path, subject, phase, ['rr_interval', 'RR', 'rr', 'interval'])

        if not path_eeg or not path_rr:
            print("-> Skip (Missing)")
            continue

        try:
            df_eeg = pd.read_csv(path_eeg)
            df_rr = pd.read_csv(path_rr)

            eeg_data, eeg_col = get_column_data(df_eeg, ['FCz', 'fcz'])
            rr_vals, rr_name = smart_extract_rr_data(df_rr)

            if eeg_data is None or rr_vals is None:
                print("-> Skip (No Data)")
                continue

            eeg_data = eeg_data.fillna(method='ffill').fillna(method='bfill')

            # LF計算 (Sensitive)
            lf_data, rr_raw_data = calculate_sensitive_lf(rr_vals)

            if lf_data is None:
                print("-> LF Calc Failed")
                continue

            # 描画
            min_len = min(len(eeg_data), len(lf_data))
            eeg_vis = eeg_data.iloc[:min_len]
            lf_vis = lf_data.iloc[:min_len]
            rr_vis = rr_raw_data.iloc[:min_len] # 背景用

            fig, ax1 = plt.subplots(figsize=(12, 5))

            # 1. 背景に薄く「生のRR間隔(Raw)」を描画 -> これが一番ギザギザ
            ax1.plot(min_max_norm(rr_vis), color='gray', alpha=0.15, label='Raw RR Interval')

            # 2. EEG (青)
            ax1.plot(min_max_norm(eeg_vis), color='blue', label='EEG (FCz)', alpha=0.8, linewidth=1.0)
            ax1.set_ylabel('EEG Power', color='blue')

            # 3. LF (オレンジ) - 窓を短くしたので以前よりギザギザなはず
            ax2 = ax1.twinx()
            ax2.plot(min_max_norm(lf_vis), color='orange', label='HRV (Sensitive LF)', alpha=0.9, linewidth=1.5)
            ax2.set_ylabel('LF Power', color='orange')

            plt.title(f"{subject} - {phase}")

            # 凡例整理
            lines1, labels1 = ax1.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

            plt.savefig(os.path.join(output_dir, f"{subject}_{phase}.png"))
            plt.close()

            print("✅ OK")
            success_count += 1

        except Exception as e:
            print(f"-> Error: {e}")

if success_count > 0:
    shutil.make_archive('Final_Results_Sensitive', 'zip', output_dir)
    from google.colab import files
    files.download('Final_Results_Sensitive.zip')

🚀 解析開始 (高感度LFモード)

[ishida - rest] ✅ OK
[ishida - boredom] ✅ OK
[ishida - flow] ✅ OK
[ishida - flow_ultra] ✅ OK
[ishida - overload] ✅ OK
[yamamoto - rest] ✅ OK
[yamamoto - boredom] ✅ OK
[yamamoto - flow] ✅ OK
[yamamoto - flow_ultra] ✅ OK
[yamamoto - overload] ✅ OK
[ohashi - rest] ✅ OK
[ohashi - boredom] ✅ OK
[ohashi - flow] ✅ OK
[ohashi - flow_ultra] ✅ OK
[ohashi - overload] ✅ OK
[miyake - rest] ✅ OK
[miyake - boredom] ✅ OK
[miyake - flow] ✅ OK
[miyake - flow_ultra] ✅ OK
[miyake - overload] ✅ OK
[mitsuhashi - rest] ✅ OK
[mitsuhashi - boredom] ✅ OK
[mitsuhashi - flow] ✅ OK
[mitsuhashi - flow_ultra] ✅ OK
[mitsuhashi - overload] ✅ OK
[ko - rest] ✅ OK
[ko - boredom] ✅ OK
[ko - flow] ✅ OK
[ko - flow_ultra] ✅ OK
[ko - overload] ✅ OK
[hayato - rest] ✅ OK
[hayato - boredom] ✅ OK
[hayato - flow] ✅ OK
[hayato - flow_ultra] ✅ OK
[hayato - overload] ✅ OK
[chika - rest] ✅ OK
[chika - boredom] ✅ OK
[chika - flow] ✅ OK
[chika - flow_ultra] ✅ OK
[chika - overload] ✅ OK
[nagomi - rest] ✅ OK
[nagomi - b

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# =============================================================================
# Project: Flow State EEG & HRV Analysis
# Mode: Raw Sensitivity (No Smoothing, High-Resolution LF)
# =============================================================================

import sys
import os
import glob
import shutil
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.interpolate import interp1d
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive

# 警告メッセージを非表示
warnings.filterwarnings('ignore')

# ---------------------------------------------------------
# 1. 設定エリア
# ---------------------------------------------------------

# Google Drive マウント
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# データ格納親フォルダ
base_search_path = "/content/drive/MyDrive/csv"

# 分析対象リスト
subject_list = [
    'ishida', 'yamamoto', 'ohashi', 'miyake', 'mitsuhashi',
    'ko', 'hayato', 'chika', 'nagomi', 'mio'
]

# 分析フェーズリスト
phase_list = ['rest', 'boredom', 'flow', 'flow_ultra', 'overload']

# 出力フォルダ名
output_dir = "Flow_Analysis_Results_Raw"

# 以前の結果があれば削除して作り直す
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)


# ---------------------------------------------------------
# 2. 関数定義
# ---------------------------------------------------------

def find_file_fuzzy(base_path, subject, phase, keywords):
    """
    指定されたキーワードを含むファイルをフォルダ階層下から検索
    """
    search_pattern = os.path.join(base_path, "**", f"*{subject}*{phase}*.csv")
    candidates = glob.glob(search_pattern, recursive=True)
    for path in candidates:
        # ファイル名にキーワードが含まれるかチェック
        if any(k.lower() in os.path.basename(path).lower() for k in keywords):
            return path
    return None

def get_column_data(df, target_names):
    """
    データフレームから対象のカラムを柔軟に取得
    """
    df.columns = [str(c).strip() for c in df.columns]
    for col in df.columns:
        for target in target_names:
            if col.lower() == target.lower():
                return df[col], col
    return None, None

def smart_extract_rr_data(df):
    """
    RR間隔データを自動特定（列名または数値範囲から判断）
    """
    # 1. 列名で検索
    series, name = get_column_data(df, ['rr_interval', 'rr', 'RR', 'interval', 'R-R', 'bpm'])
    if series is not None: return series, name

    # 2. 数値範囲で検索 (300ms ~ 1500ms)
    for col in df.columns:
        try:
            val = pd.to_numeric(df[col], errors='coerce').dropna()
            if len(val) < 10: continue
            mean_val = val.mean()
            # ms単位の場合
            if 300 < mean_val < 1500: return val, f"{col} (Auto-detect ms)"
            # 秒単位の場合 (1000倍する)
            if 0.3 < mean_val < 1.5: return val * 1000, f"{col} (Auto-detect sec)"
        except: continue
    return None, None

def calculate_lf_high_res(rr_data_ms):
    """
    RR間隔からLF成分を計算（高解像度設定）
    Window: 60s, Step: 1s
    """
    try:
        # 外れ値除去 (300ms ~ 2000ms以外を除外)
        rr_data = rr_data_ms[(rr_data_ms > 300) & (rr_data_ms < 2000)]
        if len(rr_data) < 10: return None

        # 時間軸の作成とリサンプリング (4Hz)
        t_cum = np.cumsum(rr_data) / 1000.0
        t_cum = t_cum - t_cum.iloc[0]

        fs_interp = 4.0
        t_interp = np.arange(0, t_cum.iloc[-1], 1/fs_interp)

        # 線形補間 (Linear) で変動の鋭さを維持
        f_int = interp1d(t_cum, rr_data, kind='linear', fill_value="extrapolate")
        rr_interp = f_int(t_interp)

        # スライディングウィンドウ設定
        window_sec = 60 # LF計算に必要な最低幅
        step_sec = 1    # 1秒ごとに計算（高解像度）

        nperseg = int(window_sec * fs_interp)
        step_samples = int(step_sec * fs_interp)

        if len(rr_interp) < nperseg: return None

        lf_vals = []
        t_vals = []

        for i in range(0, len(rr_interp) - nperseg, step_samples):
            segment = rr_interp[i : i + nperseg]

            # ウェルチ法でPSD計算
            freqs, psd = signal.welch(segment, fs=fs_interp, nperseg=nperseg)

            # LF帯域 (0.04-0.15Hz) のパワー積分
            lf_pow = np.trapz(psd[(freqs >= 0.04) & (freqs <= 0.15)], freqs[(freqs >= 0.04) & (freqs <= 0.15)])

            lf_vals.append(lf_pow)
            t_vals.append(t_interp[i + nperseg//2])

        # 元の時間軸に合わせて補間
        x_target = np.linspace(0, t_cum.iloc[-1], len(rr_data_ms))
        if len(t_vals) > 1:
            f_res = interp1d(t_vals, lf_vals, kind='linear', fill_value="extrapolate")
            return pd.Series(f_res(x_target), index=rr_data_ms.index)
        return None

    except Exception:
        return None

def min_max_norm(series):
    """0-1正規化"""
    if len(series) == 0: return series
    scaler = MinMaxScaler()
    return scaler.fit_transform(series.values.reshape(-1, 1)).flatten()


# ---------------------------------------------------------
# 3. メイン実行ループ
# ---------------------------------------------------------
print(f"🚀 解析を開始します...")
print(f"   対象: {len(subject_list)}名 x {len(phase_list)}フェーズ")
print(f"   モード: Smoothing OFF / LF Step 1s\n")

success_count = 0
skip_count = 0

for subject in subject_list:
    for phase in phase_list:
        print(f"[{subject} - {phase}]", end=" ")

        # ファイル検索
        path_eeg = find_file_fuzzy(base_search_path, subject, phase, ['PLI', 'pli'])
        path_rr = find_file_fuzzy(base_search_path, subject, phase, ['rr_interval', 'RR', 'rr', 'interval'])

        if not path_eeg or not path_rr:
            print("-> ⚠️ Skip (File not found)")
            skip_count += 1
            continue

        try:
            # データ読み込み
            df_eeg = pd.read_csv(path_eeg)
            df_rr = pd.read_csv(path_rr)

            # カラム特定
            eeg_data, eeg_col = get_column_data(df_eeg, ['FCz', 'fcz'])
            rr_vals, rr_name = smart_extract_rr_data(df_rr)

            if eeg_data is None or rr_vals is None:
                print("-> ⚠️ Skip (Column not found)")
                skip_count += 1
                continue

            # 欠損処理
            eeg_data = eeg_data.fillna(method='ffill').fillna(method='bfill')

            # LF計算
            lf_data = calculate_lf_high_res(rr_vals)

            if lf_data is None:
                print("-> ❌ LF Calc Failed")
                skip_count += 1
                continue

            # 描画 (スムージングなし)
            min_len = min(len(eeg_data), len(lf_data))
            eeg_vis = eeg_data.iloc[:min_len]
            lf_vis = lf_data.iloc[:min_len]

            fig, ax1 = plt.subplots(figsize=(12, 5))

            # EEG (青)
            ax1.plot(min_max_norm(eeg_vis), color='blue', label='EEG (FCz)', alpha=0.6, linewidth=0.8)
            ax1.set_ylabel('EEG Power (Normalized)', color='blue')

            # HRV (オレンジ)
            ax2 = ax1.twinx()
            ax2.plot(min_max_norm(lf_vis), color='orange', label='HRV (LF)', alpha=0.9, linewidth=1.2)
            ax2.set_ylabel('LF Power (Normalized)', color='orange')

            plt.title(f"Subject: {subject} | Phase: {phase}")

            # 凡例
            lines1, labels1 = ax1.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

            # 保存
            save_path = os.path.join(output_dir, f"{subject}_{phase}.png")
            plt.savefig(save_path)
            plt.close()

            print("✅ OK")
            success_count += 1

        except Exception as e:
            print(f"-> ❌ Error: {e}")
            skip_count += 1

# ---------------------------------------------------------
# 4. 終了処理 (Zip作成とダウンロード)
# ---------------------------------------------------------
if success_count > 0:
    print(f"\n🎉 完了しました！ (成功: {success_count}, スキップ/失敗: {skip_count})")
    zip_filename = 'final_result_raw'
    shutil.make_archive(zip_filename, 'zip', output_dir)
    from google.colab import files
    files.download(f'{zip_filename}.zip')
    print(f"⬇️ '{zip_filename}.zip' のダウンロードが開始されます。")
else:
    print("\n⚠️ 画像が作成されませんでした。")

🚀 解析を開始します...
   対象: 10名 x 5フェーズ
   モード: Smoothing OFF / LF Step 1s

[ishida - rest] ✅ OK
[ishida - boredom] ✅ OK
[ishida - flow] ✅ OK
[ishida - flow_ultra] ✅ OK
[ishida - overload] ✅ OK
[yamamoto - rest] ✅ OK
[yamamoto - boredom] ✅ OK
[yamamoto - flow] ✅ OK
[yamamoto - flow_ultra] ✅ OK
[yamamoto - overload] ✅ OK
[ohashi - rest] ✅ OK
[ohashi - boredom] ✅ OK
[ohashi - flow] ✅ OK
[ohashi - flow_ultra] ✅ OK
[ohashi - overload] ✅ OK
[miyake - rest] ✅ OK
[miyake - boredom] ✅ OK
[miyake - flow] ✅ OK
[miyake - flow_ultra] ✅ OK
[miyake - overload] ✅ OK
[mitsuhashi - rest] ✅ OK
[mitsuhashi - boredom] ✅ OK
[mitsuhashi - flow] ✅ OK
[mitsuhashi - flow_ultra] ✅ OK
[mitsuhashi - overload] ✅ OK
[ko - rest] ✅ OK
[ko - boredom] ✅ OK
[ko - flow] ✅ OK
[ko - flow_ultra] ✅ OK
[ko - overload] ✅ OK
[hayato - rest] ✅ OK
[hayato - boredom] ✅ OK
[hayato - flow] ✅ OK
[hayato - flow_ultra] ✅ OK
[hayato - overload] ✅ OK
[chika - rest] ✅ OK
[chika - boredom] ✅ OK
[chika - flow] ✅ OK
[chika - flow_ultra] ✅ OK
[chika

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️ 'final_result_raw.zip' のダウンロードが開始されます。


In [ ]:
# =============================================================================
# 【画像結合】50枚のグラフを 10行x5列 の行列画像にするコード
# =============================================================================

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

# --- 設定 ---

# 画像が入っているフォルダ (直前のコードの出力先)
input_dir = "Flow_Analysis_Results_Raw"

# 保存するファイル名
output_filename = "Combined_Matrix_5x10.png"

# リスト定義 (並び順用)
subject_list = [
    'ishida', 'yamamoto', 'ohashi', 'miyake', 'mitsuhashi',
    'ko', 'hayato', 'chika', 'nagomi', 'mio'
]

phase_list = ['rest', 'boredom', 'flow', 'flow_ultra', 'overload']

# --- 結合処理 ---

print("🖼️ 画像を結合中...")

# 1. 巨大なキャンバスを作成 (横25インチ x 縦40インチ)
# ※グラフの縦横比に合わせて figsize を調整してください
fig, axes = plt.subplots(nrows=10, ncols=5, figsize=(25, 40))

# 余白の調整 (wspace=横の間隔, hspace=縦の間隔)
plt.subplots_adjust(wspace=0.05, hspace=0.1)

for i, subject in enumerate(subject_list):
    for j, phase in enumerate(phase_list):

        ax = axes[i, j]

        # 読み込むファイル名
        # ※もしファイル名生成ルールが違う場合はここを修正
        filename = f"{subject}_{phase}.png"
        filepath = os.path.join(input_dir, filename)

        # 画像が存在するかチェックして表示
        if os.path.exists(filepath):
            img = mpimg.imread(filepath)
            ax.imshow(img)
        else:
            # 画像がない場合は空白にしつつメッセージ
            ax.text(0.5, 0.5, "No Data", ha='center', va='center', fontsize=20, color='gray')

        # グラフ自体の枠線や目盛りを消す (画像の中に既にあるため)
        ax.axis('off')

        # --- ラベル付け (卒論用に見やすく) ---

        # 1行目だけに「フェーズ名」を上部に表示
        if i == 0:
            ax.set_title(phase.upper(), fontsize=24, weight='bold', pad=20)

        # 1列目だけに「被験者名」を左側に表示
        if j == 0:
            # 左端(-0.1の位置)に、90度回転させて名前を表示
            ax.text(-0.1, 0.5, subject.upper(), transform=ax.transAxes,
                    rotation=90, va='center', ha='right', fontsize=24, weight='bold')

# --- 保存 ---
print("💾 高画質で保存中... (少し時間がかかります)")
plt.savefig(output_filename, dpi=150, bbox_inches='tight') # dpi=300だと重すぎる場合は150に
plt.close()

print(f"✅ 完了しました: {output_filename}")

# ダウンロード
from google.colab import files
files.download(output_filename)

🖼️ 画像を結合中...
💾 高画質で保存中... (少し時間がかかります)
✅ 完了しました: Combined_Matrix_5x10.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# =============================================================================
# 【画像結合】5人ずつ2枚に分割 (5行x5列 x 2ファイル)
#  見出し(Phase/Name)なし版
# =============================================================================

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import math

# --- 設定 ---

input_dir = "Flow_Analysis_Results_Raw"

# 全リスト
full_subject_list = [
    'ishida', 'yamamoto', 'ohashi', 'miyake', 'mitsuhashi',
    'ko', 'hayato', 'chika', 'nagomi', 'mio'
]

phase_list = ['rest', 'boredom', 'flow', 'flow_ultra', 'overload']

# 5人ずつに分割
groups = [
    full_subject_list[:5],  # Part 1 (前半5人)
    full_subject_list[5:]   # Part 2 (後半5人)
]

output_filenames = ["Combined_Part1_Top5.png", "Combined_Part2_Bottom5.png"]

# --- 結合処理 ---

print("🖼️ 画像結合を開始します (見出しなし / 5x5分割)...")

for group_idx, current_subjects in enumerate(groups):

    # 5行 x 5列 のキャンバス
    # figsize=(幅, 高さ) -> 20, 15 くらいがA4に貼りやすい比率です
    fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(20, 15))

    # グラフ間の隙間調整 (0にするとくっつきます。適宜調整してください)
    plt.subplots_adjust(wspace=0.05, hspace=0.1)

    for i, subject in enumerate(current_subjects):
        for j, phase in enumerate(phase_list):

            ax = axes[i, j]

            # 画像読み込み
            filename = f"{subject}_{phase}.png"
            filepath = os.path.join(input_dir, filename)

            if os.path.exists(filepath):
                img = mpimg.imread(filepath)
                ax.imshow(img)
            else:
                # 画像がない場合
                ax.text(0.5, 0.5, "No Data", ha='center', va='center', color='gray')

            # 枠線・目盛り・ラベルを全て消す (純粋に画像を貼るだけ)
            ax.axis('off')

    # 保存
    save_name = output_filenames[group_idx]
    print(f"💾 保存中: {save_name} ...")

    # dpi=300 で高画質保存 (論文用)
    plt.savefig(save_name, dpi=300, bbox_inches='tight', pad_inches=0.1)
    plt.close()

print("\n✅ 全ての処理が完了しました。")

# ダウンロード
from google.colab import files
for f in output_filenames:
    if os.path.exists(f):
        files.download(f)

🖼️ 画像結合を開始します (見出しなし / 5x5分割)...
💾 保存中: Combined_Part1_Top5.png ...
💾 保存中: Combined_Part2_Bottom5.png ...

✅ 全ての処理が完了しました。


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# =============================================================================
# 【画像結合】5人×5フェーズ (2分割)
#  ★改良版: 余白を極限まで削除し、グラフを最大化する
# =============================================================================

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

# --- 設定 ---

input_dir = "Flow_Analysis_Results_Raw"

# 全リスト
full_subject_list = [
    'ishida', 'yamamoto', 'ohashi', 'miyake', 'mitsuhashi',
    'ko', 'hayato', 'chika', 'nagomi', 'mio'
]

phase_list = ['rest', 'boredom', 'flow', 'flow_ultra', 'overload']

# 5人ずつに分割
groups = [
    full_subject_list[:5],  # Part 1 (前半5人)
    full_subject_list[5:]   # Part 2 (後半5人)
]

output_filenames = ["Combined_Part1_Top5_Max.png", "Combined_Part2_Bottom5_Max.png"]

# --- 結合処理 ---

print("🖼️ 画像結合を開始します (余白ゼロ・最大化モード)...")

for group_idx, current_subjects in enumerate(groups):

    # ★ポイント1: figsizeの比率を調整
    # 元のグラフが横長なので、横幅を広く取ります (25インチ x 10インチ)
    # これにより、個々のグラフが押しつぶされずにピタリと収まります
    fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(25, 10),
                             gridspec_kw={'wspace': 0, 'hspace': 0}) # ★ポイント2: グラフ間の隙間を0にする

    # キャンバス自体の余白も削除
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1)

    for i, subject in enumerate(current_subjects):
        for j, phase in enumerate(phase_list):

            ax = axes[i, j]

            # 画像読み込み
            filename = f"{subject}_{phase}.png"
            filepath = os.path.join(input_dir, filename)

            if os.path.exists(filepath):
                img = mpimg.imread(filepath)
                # aspect='auto' にすると、枠に合わせて画像を多少伸縮させてでも埋めます
                # 画像の縦横比を厳密に維持したい場合は 'equal' に戻してください
                ax.imshow(img, aspect='auto')
            else:
                ax.text(0.5, 0.5, "No Data", ha='center', va='center', color='gray')

            # 軸をオフ
            ax.axis('off')

    # 保存
    save_name = output_filenames[group_idx]
    print(f"💾 保存中: {save_name} ...")

    # ★ポイント3: pad_inches=0 で外側の余白も完全カット
    plt.savefig(save_name, dpi=300, bbox_inches='tight', pad_inches=0)
    plt.close()

print("\n✅ 完了しました。")

# ダウンロード
from google.colab import files
for f in output_filenames:
    if os.path.exists(f):
        files.download(f)

🖼️ 画像結合を開始します (余白ゼロ・最大化モード)...
💾 保存中: Combined_Part1_Top5_Max.png ...
💾 保存中: Combined_Part2_Bottom5_Max.png ...

✅ 完了しました。


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# =============================================================================
# 【極太・高視認性版】EEG & HRV 解析コード
# =============================================================================
# 変更点:
# 1. linewidth (線の太さ) を大幅にアップ
# 2. fontsize (文字サイズ) をアップし、fontweight='bold' (太字) に設定
# =============================================================================

import sys
import os
import glob
import shutil
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.interpolate import interp1d
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive

warnings.filterwarnings('ignore')

# --- 設定 ---
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

base_search_path = "/content/drive/MyDrive/csv"

subject_list = [
    'ishida', 'yamamoto', 'ohashi', 'miyake', 'mitsuhashi',
    'ko', 'hayato', 'chika', 'nagomi', 'mio'
]

phase_list = ['rest', 'boredom', 'flow', 'flow_ultra', 'overload']

output_dir = "Flow_Analysis_Results_Bold" # 出力先を変更
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

# --- 共通関数 (変更なし) ---
def find_file_fuzzy(base_path, subject, phase, keywords):
    search_pattern = os.path.join(base_path, "**", f"*{subject}*{phase}*.csv")
    candidates = glob.glob(search_pattern, recursive=True)
    for path in candidates:
        if any(k.lower() in os.path.basename(path).lower() for k in keywords):
            return path
    return None

def get_column_data(df, target_names):
    df.columns = [str(c).strip() for c in df.columns]
    for col in df.columns:
        for target in target_names:
            if col.lower() == target.lower():
                return df[col], col
    return None, None

def smart_extract_rr_data(df):
    series, name = get_column_data(df, ['rr_interval', 'rr', 'RR', 'interval', 'R-R', 'bpm'])
    if series is not None: return series, name
    for col in df.columns:
        try:
            val = pd.to_numeric(df[col], errors='coerce').dropna()
            if len(val) < 10: continue
            mean_val = val.mean()
            if 300 < mean_val < 1500: return val, f"{col} (Auto ms)"
            if 0.3 < mean_val < 1.5: return val * 1000, f"{col} (Auto sec)"
        except: continue
    return None, None

def calculate_lf_high_res(rr_data_ms):
    try:
        rr_data = rr_data_ms[(rr_data_ms > 300) & (rr_data_ms < 2000)]
        if len(rr_data) < 10: return None
        t_cum = np.cumsum(rr_data) / 1000.0
        t_cum = t_cum - t_cum.iloc[0]
        fs_interp = 4.0
        t_interp = np.arange(0, t_cum.iloc[-1], 1/fs_interp)
        f_int = interp1d(t_cum, rr_data, kind='linear', fill_value="extrapolate")
        rr_interp = f_int(t_interp)

        window_sec = 60
        step_sec = 1
        nperseg = int(window_sec * fs_interp)
        step_samples = int(step_sec * fs_interp)
        if len(rr_interp) < nperseg: return None

        lf_vals = []
        t_vals = []
        for i in range(0, len(rr_interp) - nperseg, step_samples):
            segment = rr_interp[i : i + nperseg]
            freqs, psd = signal.welch(segment, fs=fs_interp, nperseg=nperseg)
            lf_pow = np.trapz(psd[(freqs >= 0.04) & (freqs <= 0.15)], freqs[(freqs >= 0.04) & (freqs <= 0.15)])
            lf_vals.append(lf_pow)
            t_vals.append(t_interp[i + nperseg//2])

        x_target = np.linspace(0, t_cum.iloc[-1], len(rr_data_ms))
        if len(t_vals) > 1:
            f_res = interp1d(t_vals, lf_vals, kind='linear', fill_value="extrapolate")
            return pd.Series(f_res(x_target), index=rr_data_ms.index)
        return None
    except: return None

def min_max_norm(series):
    if len(series) == 0: return series
    scaler = MinMaxScaler()
    return scaler.fit_transform(series.values.reshape(-1, 1)).flatten()

# --- メイン処理 ---
print(f"🚀 解析開始 (極太ライン・太文字モード)\n")

# ★全体の文字サイズを一括設定
plt.rcParams.update({'font.size': 14, 'font.weight': 'bold', 'axes.labelweight': 'bold'})

for subject in subject_list:
    for phase in phase_list:
        print(f"[{subject} - {phase}]", end=" ")

        path_eeg = find_file_fuzzy(base_search_path, subject, phase, ['PLI', 'pli'])
        path_rr = find_file_fuzzy(base_search_path, subject, phase, ['rr_interval', 'RR', 'rr', 'interval'])

        if not path_eeg or not path_rr:
            print("-> Skip")
            continue

        try:
            df_eeg = pd.read_csv(path_eeg)
            df_rr = pd.read_csv(path_rr)
            eeg_data, _ = get_column_data(df_eeg, ['FCz', 'fcz'])
            rr_vals, _ = smart_extract_rr_data(df_rr)

            if eeg_data is None or rr_vals is None: continue

            eeg_data = eeg_data.fillna(method='ffill').fillna(method='bfill')
            lf_data = calculate_lf_high_res(rr_vals)
            if lf_data is None: continue

            min_len = min(len(eeg_data), len(lf_data))
            eeg_vis = eeg_data.iloc[:min_len]
            lf_vis = lf_data.iloc[:min_len]

            fig, ax1 = plt.subplots(figsize=(12, 5))

            # ★線の太さを変更 (linewidth)
            # EEG: 青色, 太さ 2.0
            ax1.plot(min_max_norm(eeg_vis), color='blue', label='EEG', alpha=0.7, linewidth=2.0)
            ax1.set_ylabel('EEG Power', color='blue', fontsize=16, fontweight='bold')
            ax1.tick_params(axis='y', labelcolor='blue', labelsize=12)

            # LF: オレンジ, 太さ 3.0 (主役なのでより太く)
            ax2 = ax1.twinx()
            ax2.plot(min_max_norm(lf_vis), color='orange', label='HRV (LF)', alpha=1.0, linewidth=3.5)
            ax2.set_ylabel('LF Power', color='orange', fontsize=16, fontweight='bold')
            ax2.tick_params(axis='y', labelcolor='orange', labelsize=12)

            # タイトルを削除（あとで結合するなら邪魔になるため。必要ならコメントアウト解除）
            # plt.title(f"{subject} - {phase}", fontsize=18, fontweight='bold')

            # 凡例 (Legend) を大きく
            lines1, labels1 = ax1.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=14, frameon=True)

            plt.savefig(os.path.join(output_dir, f"{subject}_{phase}.png"))
            plt.close()
            print("✅ OK")

        except Exception as e:
            print(f"-> Error: {e}")

print("\n🎉 画像生成完了")

🚀 解析開始 (極太ライン・太文字モード)

[ishida - rest] ✅ OK
[ishida - boredom] ✅ OK
[ishida - flow] ✅ OK
[ishida - flow_ultra] ✅ OK
[ishida - overload] ✅ OK
[yamamoto - rest] ✅ OK
[yamamoto - boredom] ✅ OK
[yamamoto - flow] ✅ OK
[yamamoto - flow_ultra] ✅ OK
[yamamoto - overload] ✅ OK
[ohashi - rest] ✅ OK
[ohashi - boredom] ✅ OK
[ohashi - flow] ✅ OK
[ohashi - flow_ultra] ✅ OK
[ohashi - overload] ✅ OK
[miyake - rest] ✅ OK
[miyake - boredom] ✅ OK
[miyake - flow] ✅ OK
[miyake - flow_ultra] ✅ OK
[miyake - overload] ✅ OK
[mitsuhashi - rest] ✅ OK
[mitsuhashi - boredom] ✅ OK
[mitsuhashi - flow] ✅ OK
[mitsuhashi - flow_ultra] ✅ OK
[mitsuhashi - overload] ✅ OK
[ko - rest] ✅ OK
[ko - boredom] ✅ OK
[ko - flow] ✅ OK
[ko - flow_ultra] ✅ OK
[ko - overload] ✅ OK
[hayato - rest] ✅ OK
[hayato - boredom] ✅ OK
[hayato - flow] ✅ OK
[hayato - flow_ultra] ✅ OK
[hayato - overload] ✅ OK
[chika - rest] ✅ OK
[chika - boredom] ✅ OK
[chika - flow] ✅ OK
[chika - flow_ultra] ✅ OK
[chika - overload] ✅ OK
[nagomi - rest] ✅ OK
[nagomi

In [ ]:
# =============================================================================
# 【題名復活・極太版】EEG & HRV 解析コード
# =============================================================================

import sys
import os
import glob
import shutil
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.interpolate import interp1d
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive

warnings.filterwarnings('ignore')

# --- 設定 ---
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

base_search_path = "/content/drive/MyDrive/csv"

subject_list = [
    'ishida', 'yamamoto', 'ohashi', 'miyake', 'mitsuhashi',
    'ko', 'hayato', 'chika', 'nagomi', 'mio'
]

phase_list = ['rest', 'boredom', 'flow', 'flow_ultra', 'overload']

# 出力先を新しい名前に
output_dir = "Flow_Results_Bold_Title"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

# --- 共通関数 ---
def find_file_fuzzy(base_path, subject, phase, keywords):
    search_pattern = os.path.join(base_path, "**", f"*{subject}*{phase}*.csv")
    candidates = glob.glob(search_pattern, recursive=True)
    for path in candidates:
        if any(k.lower() in os.path.basename(path).lower() for k in keywords):
            return path
    return None

def get_column_data(df, target_names):
    df.columns = [str(c).strip() for c in df.columns]
    for col in df.columns:
        for target in target_names:
            if col.lower() == target.lower():
                return df[col], col
    return None, None

def smart_extract_rr_data(df):
    series, name = get_column_data(df, ['rr_interval', 'rr', 'RR', 'interval', 'R-R', 'bpm'])
    if series is not None: return series, name
    for col in df.columns:
        try:
            val = pd.to_numeric(df[col], errors='coerce').dropna()
            if len(val) < 10: continue
            mean_val = val.mean()
            if 300 < mean_val < 1500: return val, f"{col} (Auto ms)"
            if 0.3 < mean_val < 1.5: return val * 1000, f"{col} (Auto sec)"
        except: continue
    return None, None

def calculate_lf_high_res(rr_data_ms):
    try:
        rr_data = rr_data_ms[(rr_data_ms > 300) & (rr_data_ms < 2000)]
        if len(rr_data) < 10: return None
        t_cum = np.cumsum(rr_data) / 1000.0
        t_cum = t_cum - t_cum.iloc[0]
        fs_interp = 4.0
        t_interp = np.arange(0, t_cum.iloc[-1], 1/fs_interp)
        f_int = interp1d(t_cum, rr_data, kind='linear', fill_value="extrapolate")
        rr_interp = f_int(t_interp)

        window_sec = 60
        step_sec = 1
        nperseg = int(window_sec * fs_interp)
        step_samples = int(step_sec * fs_interp)
        if len(rr_interp) < nperseg: return None

        lf_vals = []
        t_vals = []
        for i in range(0, len(rr_interp) - nperseg, step_samples):
            segment = rr_interp[i : i + nperseg]
            freqs, psd = signal.welch(segment, fs=fs_interp, nperseg=nperseg)
            lf_pow = np.trapz(psd[(freqs >= 0.04) & (freqs <= 0.15)], freqs[(freqs >= 0.04) & (freqs <= 0.15)])
            lf_vals.append(lf_pow)
            t_vals.append(t_interp[i + nperseg//2])

        x_target = np.linspace(0, t_cum.iloc[-1], len(rr_data_ms))
        if len(t_vals) > 1:
            f_res = interp1d(t_vals, lf_vals, kind='linear', fill_value="extrapolate")
            return pd.Series(f_res(x_target), index=rr_data_ms.index)
        return None
    except: return None

def min_max_norm(series):
    if len(series) == 0: return series
    scaler = MinMaxScaler()
    return scaler.fit_transform(series.values.reshape(-1, 1)).flatten()

# --- メイン処理 ---
print(f"🚀 解析開始 (題名付き・極太モード)\n")

# フォント設定
plt.rcParams.update({'font.size': 14, 'font.weight': 'bold', 'axes.labelweight': 'bold'})

for subject in subject_list:
    for phase in phase_list:
        print(f"[{subject} - {phase}]", end=" ")

        path_eeg = find_file_fuzzy(base_search_path, subject, phase, ['PLI', 'pli'])
        path_rr = find_file_fuzzy(base_search_path, subject, phase, ['rr_interval', 'RR', 'rr', 'interval'])

        if not path_eeg or not path_rr:
            print("-> Skip")
            continue

        try:
            df_eeg = pd.read_csv(path_eeg)
            df_rr = pd.read_csv(path_rr)
            eeg_data, _ = get_column_data(df_eeg, ['FCz', 'fcz'])
            rr_vals, _ = smart_extract_rr_data(df_rr)

            if eeg_data is None or rr_vals is None: continue

            eeg_data = eeg_data.fillna(method='ffill').fillna(method='bfill')
            lf_data = calculate_lf_high_res(rr_vals)
            if lf_data is None: continue

            min_len = min(len(eeg_data), len(lf_data))
            eeg_vis = eeg_data.iloc[:min_len]
            lf_vis = lf_data.iloc[:min_len]

            fig, ax1 = plt.subplots(figsize=(12, 5))

            # 線を太く
            ax1.plot(min_max_norm(eeg_vis), color='blue', label='EEG', alpha=0.7, linewidth=2.0)
            ax1.set_ylabel('EEG Power', color='blue', fontsize=16, fontweight='bold')
            ax1.tick_params(axis='y', labelcolor='blue', labelsize=12)

            # LFをさらに太く
            ax2 = ax1.twinx()
            ax2.plot(min_max_norm(lf_vis), color='orange', label='HRV (LF)', alpha=1.0, linewidth=3.5)
            ax2.set_ylabel('LF Power', color='orange', fontsize=16, fontweight='bold')
            ax2.tick_params(axis='y', labelcolor='orange', labelsize=12)

            # ★タイトルを復活 (大きく太く)
            title_str = f"{subject.upper()} - {phase.upper()}"
            plt.title(title_str, fontsize=20, fontweight='bold', pad=15)

            lines1, labels1 = ax1.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=14, frameon=True)

            # タイトルが切れないように bbox_inches='tight' で保存
            plt.savefig(os.path.join(output_dir, f"{subject}_{phase}.png"), bbox_inches='tight')
            plt.close()
            print("✅ OK")

        except Exception as e:
            print(f"-> Error: {e}")

print("\n🎉 画像生成完了")

🚀 解析開始 (題名付き・極太モード)

[ishida - rest] ✅ OK
[ishida - boredom] ✅ OK
[ishida - flow] ✅ OK
[ishida - flow_ultra] ✅ OK
[ishida - overload] ✅ OK
[yamamoto - rest] ✅ OK
[yamamoto - boredom] ✅ OK
[yamamoto - flow] ✅ OK
[yamamoto - flow_ultra] ✅ OK
[yamamoto - overload] ✅ OK
[ohashi - rest] ✅ OK
[ohashi - boredom] ✅ OK
[ohashi - flow] ✅ OK
[ohashi - flow_ultra] ✅ OK
[ohashi - overload] ✅ OK
[miyake - rest] ✅ OK
[miyake - boredom] ✅ OK
[miyake - flow] ✅ OK
[miyake - flow_ultra] ✅ OK
[miyake - overload] ✅ OK
[mitsuhashi - rest] ✅ OK
[mitsuhashi - boredom] ✅ OK
[mitsuhashi - flow] ✅ OK
[mitsuhashi - flow_ultra] ✅ OK
[mitsuhashi - overload] ✅ OK
[ko - rest] ✅ OK
[ko - boredom] ✅ OK
[ko - flow] ✅ OK
[ko - flow_ultra] ✅ OK
[ko - overload] ✅ OK
[hayato - rest] ✅ OK
[hayato - boredom] ✅ OK
[hayato - flow] ✅ OK
[hayato - flow_ultra] ✅ OK
[hayato - overload] ✅ OK
[chika - rest] ✅ OK
[chika - boredom] ✅ OK
[chika - flow] ✅ OK
[chika - flow_ultra] ✅ OK
[chika - overload] ✅ OK
[nagomi - rest] ✅ OK
[nagomi -

In [ ]:
# =============================================================================
# 【画像結合】題名付き画像の結合 (最大化・余白微調整)
# =============================================================================

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

# ★ 題名付きのフォルダを指定
input_dir = "Flow_Results_Bold_Title"

full_subject_list = [
    'ishida', 'yamamoto', 'ohashi', 'miyake', 'mitsuhashi',
    'ko', 'hayato', 'chika', 'nagomi', 'mio'
]
phase_list = ['rest', 'boredom', 'flow', 'flow_ultra', 'overload']

groups = [full_subject_list[:5], full_subject_list[5:]]
output_filenames = ["Combined_Titled_Part1.png", "Combined_Titled_Part2.png"]

print("🖼️ 題名付き画像を結合中...")

for group_idx, current_subjects in enumerate(groups):
    # figsize=(25, 12) 少し縦を長くして、タイトルが入る余裕を持たせます
    fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(25, 12),
                             gridspec_kw={'wspace': 0, 'hspace': 0.1})
                             # hspace=0.1: 題名が上のグラフと被らないようにわずかに隙間を空けます

    # 外枠の余白削除
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1)

    for i, subject in enumerate(current_subjects):
        for j, phase in enumerate(phase_list):
            ax = axes[i, j]
            filename = f"{subject}_{phase}.png"
            filepath = os.path.join(input_dir, filename)

            if os.path.exists(filepath):
                img = mpimg.imread(filepath)
                ax.imshow(img, aspect='auto')
            else:
                ax.text(0.5, 0.5, "No Data", ha='center', va='center', color='gray')
            ax.axis('off')

    save_name = output_filenames[group_idx]
    # pad_inches=0.1 わずかに外側に余白を残してタイトル切れ防止
    plt.savefig(save_name, dpi=300, bbox_inches='tight', pad_inches=0.1)
    plt.close()
    print(f"💾 保存完了: {save_name}")

# ダウンロード
from google.colab import files
for f in output_filenames:
    if os.path.exists(f):
        files.download(f)

🖼️ 題名付き画像を結合中...
💾 保存完了: Combined_Titled_Part1.png
💾 保存完了: Combined_Titled_Part2.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>